# Assignment 5 Scrapping Data
### Kevin Williams 

Ask prof for quick check 


# Imports 

In [1]:
import pandas as pd
from requests_html import HTMLSession
import os
import re
from time import sleep
from tqdm.notebook import tqdm # progress bar
from NEAR_regex import NEAR_regex 
from bs4 import BeautifulSoup
import numpy as np


0
1
0
0
0
1
0
0
1


In [4]:
ccm = pd.read_stata('input/2007_inv_and_tech.dta')

# create 5 new risk variables
ccm_with_risk = ccm.assign(
    govt_risk1        = np.nan,
    govt_risk2        = np.nan,
    govt_risk3        = np.nan,
    currency_risk     = np.nan,
    reputational_risk = np.nan,
)

risk_columns = ccm_with_risk[['CoName','govt_risk1','govt_risk2','govt_risk1','currency_risk','reputational_risk']]
# risk_columns


# Searching Risks and Parsing 

#### Golden Rules: Parsing 

1. Use html tags to change/remove unneeded sections. 
2. Remove html tags, and turn the document into a pure text string.
3. Lower case everything.
4. Delete punctuation.
5. Delete all excess whitespace.
6. Now you can search/parse the text.


In [6]:
def get_ccm_Filepath(row):
    '''
    Parameters
    ----------
    row: Pandas dataFrame indicating "row of passed" df
    
    Warning / Feature
    -------
    
    This function will only work for the ccm dataFrame beacause of "FName" and thus is not flexable 
    
    Returns
    -------
    This function returns text or byte string giving the name (and the path
    if the file isn't in the current working directory) of the file to
    be opened. If name of file is not found returns false.
    '''
    if row['FName'] != '': 
        form     = row['Form'].replace('/','').replace('\\','')  
        CoName   = row['CoName'].replace('/','')
        folder   =  f'edgar_filings/{CoName}/{form}/fyear_{int(row["fyear"])}/'
        filename = row['FName'].split('/')[-1] 
        filepath = folder + filename
        return filepath
    else: 
        return False


# Disclamer !

The code below does not compile correctly. In this assignment I am having an issue with updating the risk calculations to the rows. Take a look at my test section, which has the same exact code and runs perfectly for another df. 

I was told that when I created the risk columns to use np.nan instead of 0, but either way the NEAR_regex search results are not being inputed into ccm_with_risk.

Moving forward, pay attention to my "style of coding" because for the most part I understand how to code this assignment. 

*The analysis folder will now use a static df i created to demonstrate my visualization and anlaysis skills*

In [12]:
# parse company filings and run risk searches

for index, row in tqdm(ccm_with_risk.iterrows(),total=len(ccm_with_risk),desc='Parsing and searching risks'):    
    firm               = ccm_with_risk.iloc[index]
    filepath           = get_ccm_Filepath(firm)
    
    if filepath != False:
        company_filing = open(filepath, 'r') # open Company 10-K file txt
        lower          = BeautifulSoup(company_filing.read()).get_text().lower() # removes html tags and lower case everything 
        no_punc        = re.sub(r'\W',' ',lower) # remove punctuations (\w is word chars, \W is the opposite)
        cleaned        = re.sub(r'\s+',' ',no_punc).strip() # remove excess whitespace (\s is all whitespace)
        
        # risk searches
        words = ['government','risk']
        rgx = NEAR_regex(words,max_words_between=100,partial=True)
#         ccm_with_risk.loc[index,'govt_risk1'] = len(re.findall(rgx,cleaned))
        row['govt_risk1'] = len(re.findall(rgx,cleaned))
        
        # increased max_words_between
        words = ['government','risk']
        rgx = NEAR_regex(words,max_words_between=150,partial=True)
        row['govt_risk2'] = len(re.findall(rgx,cleaned))
        
        # changed number of words  
        words = ['government','negatively','risk']
        rgx = NEAR_regex(words,max_words_between=200,partial=True)
        row['govt_risk3'] = len(re.findall(rgx,cleaned))
        
        words = ['currency','risk']
        rgx = NEAR_regex(words,max_words_between=100,partial=True)
        row['currency_risk'] = len(re.findall(rgx,cleaned))

        words = ['reputation','risk']
        rgx = NEAR_regex(words,max_words_between=100,partial=True)
        row['reputational_risk'] = len(re.findall(rgx,cleaned))
    
    else:
        #go to next row in df 
        continue

print(risk_columns) #prints appropriate risk columns




                            CoName  govt_risk1  govt_risk2  govt_risk1  \
0          ABM INDUSTRIES INC /DE/         NaN         NaN         NaN   
1                                          NaN         NaN         NaN   
2      ANALYSTS INTERNATIONAL CORP         NaN         NaN         NaN   
3                                          NaN         NaN         NaN   
4                     AUTODESK INC         NaN         NaN         NaN   
5                                          NaN         NaN         NaN   
6    AUTOMATIC DATA PROCESSING INC         NaN         NaN         NaN   
7                                          NaN         NaN         NaN   
8                      UNISYS CORP         NaN         NaN         NaN   
9                                          NaN         NaN         NaN   
10                     ACXIOM CORP         NaN         NaN         NaN   
11                                         NaN         NaN         NaN   
12                        CDI CORP   

In [11]:
risk_columns.head()
ccm.iloc[:,-5:].describe()

,CoName,govt_risk1,govt_risk2,govt_risk1,currency_risk,reputational_risk
0,ABM INDUSTRIES INC /DE/,NaN,NaN,NaN,NaN,NaN
1,,NaN,NaN,NaN,NaN,NaN
2,ANALYSTS INTERNATIONAL CORP,NaN,NaN,NaN,NaN,NaN
3,,NaN,NaN,NaN,NaN,NaN
4,AUTODESK INC,NaN,NaN,NaN,NaN,NaN


### Output ccm_with_risk data with new risk measures 

In [ ]:
# output ccm_with_risk data with new risk measures 

if not os.path.exists('output/'):
    os.makedirs('output/') 
    # save ccm_with_risk inside folder

ccm_with_risk.to_stata('output/ccm_with_risk.dta')
    

In [ ]:
# test if export was successful    
data_risk = pd.read_stata('output/ccm_with_risk.dta')
data_risk 


# Test Section

### Below tests the risk search measures to see if they are useful

In [9]:
# testing parsing, searching risks, and updating a generated row df
data = {'govt_risk1':[0, 0, 0,0,0], 'govt_risk2':[0, 0, 0,0,0], 'blank_risk':[0, 0, 0,0,0]} 
df = pd.DataFrame(data) 
# df = ccm_with_risk[:100].copy(deep=True).drop(columns=['level_0', 'index'])

# print(df)
for index, row in tqdm(df.iterrows(),desc='Parsing and searching risks '):
    i = index
    index              = ccm_with_risk.iloc[index]
    filepath           = get_ccm_Filepath(index)
    
    if filepath != False:
        company_filing = open(filepath, 'r') # open Company 10-K file txt
        lower          = BeautifulSoup(company_filing.read()).get_text().lower() # removes html tags and lower case everything 
        no_punc        = re.sub(r'\W',' ',lower) # remove punctuations (\w is word chars, \W is the opposite)
        cleaned        = re.sub(r'\s+',' ',no_punc).strip() # remove excess whitespace (\s is all whitespace)
        company_filing.close()
        
       # risk searches
        words = ['government','risk']
        rgx = NEAR_regex(words,max_words_between=100,partial=True)
        df.loc[i,'govt_risk1'] = len(re.findall(rgx,cleaned))
#         row['govt_risk1'] = len(re.findall(rgx,cleaned))
        
        # increased max_words_between
        words = ['government','risk']
        rgx = NEAR_regex(words,max_words_between=150,partial=True)
        row['govt_risk2'] = len(re.findall(rgx,cleaned))
        
        # changed number of words  
        words = ['government','negatively','risk']
        rgx = NEAR_regex(words,max_words_between=200,partial=True)
        row['blank_risk'] = len(re.findall(rgx,cleaned))
        
#         # new risk
#         words = ['currency','risk']
#         rgx = NEAR_regex(words,max_words_between=100,partial=True)
#         row['blank_risk'] = len(re.findall(rgx,cleaned))
    
    else:
        #go to next row in df 
        continue
df



,govt_risk1,govt_risk2,blank_risk
0,1,2,0
1,0,0,0
2,1,1,0
3,0,0,0
4,2,2,2


In [3]:
#testing risk search on test string 
a_string_you_have = 'interest rates pose a risk that impacts the economy. negative interest rates are not a a a risk we face.'
a_string_you_have = a_string_you_have.lower()

interestRate_risk1 = ['interest rates','risk']
rgx = NEAR_regex(interestRate_risk1,max_words_between=5,partial=False)
print('search result:' , len(re.findall(rgx,a_string_you_have)))



search result: 2


# Personal Notes - Ignore 


In [ ]:
# # This section creates two lists and runs NEAR_regex search
# key_words = ['interest rates','derivatives','inflation']
# # order of key-words should match up with risk columns 
# focal_words = ['negatively', 'impacts', 'affects', 'prices'] 
# for i in range(len(key_words)):
#     column = len(key_words) - i 
#     # able to acesss the correct column in ccm_with_risk to input risk calculation
#     focal_words.append(key_words[i])
#     rgx = NEAR_regex(focal_words,max_words_between=100,partial=True) # updates all risk columns 
#     search_hits = len(re.findall(rgx,cleaned)) 
#     ccm_with_risk.loc[row,-column] = search_hits
#     # IMPORTANT to remove the key_word
#     focal_words.remove(key_words[i]) 

# DataFrame.iterrows is a generator which yield both index and row
# for index, row in df.iterrows():
#       ifor_val = something
#       if <condition>:
#         ifor_val = something_else
#       df.at[index,row] = ifor_val

# Understand the error below for code:  df.at[index,'c1'] = s1
#ValueError: cannot index with vector containing NA / NaN values

In [ ]:
# Example of Professor define risk 

# this will look for mentions with 25 word gaps maximum
# tariff_search = ['(risk|risks|could harm|negative|negatively)',
#                 '(tariff|tariffs)',
#                 '(change|new|law|policy|policies|regulation|regulations)']